In [1]:
a = np.array([[1, 2, 3], [4, 5, 6]])
print(a)
print(a.transpose())

NameError: name 'np' is not defined

In [1]:
#%matplotlib widget
%matplotlib inline

import numpy as np
import sep
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.ndimage import median_filter

from photutils.datasets import make_100gaussians_image
from photutils.background import Background2D, MedianBackground

from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel

from plotimg import plot_images
from lvmagp.images import Image
from lvmagp.images.processors.detection import DaophotSourceDetection, SepSourceDetection
from lvmagp.focus.focusseries import ProjectionFocusSeries, PhotometryFocusSeries

from photutils.centroids import centroid_quadratic


deblend_nthresh = 1.4
num_stars = 7


#source_detection = SepSourceDetection()
source_detection = SepSourceDetection(threshold= 3.0, minarea=14.0, deblend_nthresh=deblend_nthresh)

fbase = lambda idx: f"/data/lvm/sci/agcam/east/20221028n/lvm.sci.agcam.east_{idx:08}.fits"

#off=390
#num=9

#off=392
#num=4


off=400
num=15

#off=357
#num=7

#off=178
#num=2

#off=319
#num=5

#off=124

focusum=[]
sepnum=[]
pfsnum=[]

for i in range(num):
    print(fbase(i+off))
    img_med = img_det = img_sep = Image.from_file(fbase(i+off))
    img_med = img_det = img_sep

#    img_det = await source_detection(img_det)
#    img_det.catalog.sort("peak")
#    img_det.catalog.reverse()
    #print(img_det.catalog[img_det.catalog["fwhm"] > 4.0])
    
    img_med.data = median_filter(img_med.data, size=2)
#    img_med = await source_detection(img_med)
#    img_med.catalog.sort("peak")
#    img_med.catalog.reverse()
    #print(img_med.catalog[img_med.catalog["fwhm"] > 4.0])

    print(img_sep.data.shape)

#    bkg_estimator = MedianBackground()
#    bkg = Background2D(img_med.data.astype(float), (50, 50), filter_size=(3, 3),
#                       bkg_estimator=bkg_estimator)
#    img_sep.data = img_sep.data.astype(float) - bkg.background

#    bkg_median = np.median(bkg.background)
#    bkg_std = np.std(bkg.background)
#    print(f"median {bkg_median}, std {bkg_std}")
    bkg = sep.Background(img_med.data.astype(float))
    img_sep.data = img_sep.data.astype(float) - bkg
    bkg_median = np.median(bkg)
    
    
#    kernel = make_2dgaussian_kernel(3.0, size=3)  # FWHM = 3.
#    img_sep.data = convolve(img_sep.data, kernel)

#    img_sep.data -= bkg
    img_sep = await source_detection(img_sep)
#    sources = sources[sources["ellipticity"] < 0.1]
    img_sep.catalog = img_sep.catalog[img_sep.catalog["peak"] > bkg_median*2]
#    img_sep.catalog = img_sep.catalog[img_sep.catalog["ellipticity"] < 0.35]
    img_sep.catalog.sort("flux")
    img_sep.catalog.reverse()
    #print(img_med.catalog[img_med.catalog["fwhm"] > 4.0])
    
#    print(f"det: {len(img_det.catalog)} med: {len(img_med.catalog)} sep {len(img_sep.catalog)}")
    print(img_sep.header["focusum"])
    focusum.append(img_sep.header["focusum"])
    sepnum.append(len(img_sep.catalog))

    pfs = PhotometryFocusSeries(source_detection, radius_column="fwhm", deblend_nthresh=deblend_nthresh)
    await pfs.analyse_image(img_sep, img_sep.header["focusum"])
    pfs_sum = pfs._data
    print(pfs_sum)
    pfsnum.append(pfs_sum[0]["r"])

#    print(img_sep.catalog[:num_stars]["ellipticity"])
#    print(img_sep.catalog[:num_stars]["peak"])
#    print(img_sep.catalog[:num_stars]["fwhm"])
#    print(img_sep.catalog[:num_stars]["kronrad"])
#    print(img_sep.catalog[:num_stars]["flux"])

#    sources = img_sep.catalog[:num_stars]
#    search_boxsize=15
#    ref = np.array([sources['x'], sources['y']]).transpose()
#    print(ref)
#    print(np.array(
#        [centroid_quadratic(img_sep.data, xpeak=x, ypeak=y, search_boxsize=search_boxsize) 
#            for x, y in ref]))

#    print()
    cat_extra=None
#    cat_extra=[{'x': 820, 'y': 742, 'a': 10, 'b': 10, 'theta': 0}]
    
#    plot_images([img_det, img_med, img_sep], rotate=[1,0,1], vmax=300, cat_max=42, cat_extra=cat_extra)
    plot_images([img_sep], cat_max=num_stars, cat_rest=True, cat_extra=cat_extra)
#    plot_images([img_det, img_sep], vmax=500)



print (pfs._data)

fig = plt.figure()
fig.canvas.toolbar_visible = 'fade-in-fade-out'
plt.plot(focusum, pfsnum);
img_sep.catalog[:num_stars]


#for idx, dd in enumerate(data):
#    focus = [d["focus"] for d in dd]
#    r = [d["r"] for d in dd]
#    rerr = [d["rerr"] for d in dd]

#    ax_idx = ax[idx] if len(data) > 1 else ax
#    ax_idx.set_title(images[idx].header["CAMNAME"])
#    ax_idx.errorbar(focus, r, yerr=rerr)

#    foc, err = fit_hyperbola(focus, r, rerr)
#    print(f"focus: {foc}, {err}")


ModuleNotFoundError: No module named 'plotimg'

In [ ]:

img_det.catalog.sort("fwhm")
img_det.catalog.reverse()

img_det.catalog[img_det.catalog["fwhm"] > 6.0]